The remaining unvectorized part of a spectrum calculation is the for loop in simsignals. It is not the bottleneck (eigh is), but it's significant. This notebook explores possible vectorizations.

In [3]:
import cProfile
import os
import sys
import time
import timeit

In [4]:
import numpy as np
import scipy
import sparse

In [5]:
from nmrtools.nmrmath import simsignals

In [6]:
from tests.test_simsignals import intensity_and_energy, old_compile_spectrum, H_RIOUX, H_RIOUX_SPARSE

Import type:  <class 'numpy.ndarray'>


In [7]:
I, E = intensity_and_energy(H_RIOUX_SPARSE, 3)
E

array([-491.625     , -230.96347143, -200.30588633,  -72.10564224,
         61.88341712,  195.52427327,  234.21730962,  503.375     ])

In [8]:
E_matrix = np.abs(E[:, np.newaxis] - E)
E_matrix

array([[  0.        , 260.66152857, 291.31911367, 419.51935776,
        553.50841712, 687.14927327, 725.84230962, 995.        ],
       [260.66152857,   0.        ,  30.65758509, 158.85782918,
        292.84688854, 426.48774469, 465.18078104, 734.33847143],
       [291.31911367,  30.65758509,   0.        , 128.20024409,
        262.18930345, 395.8301596 , 434.52319595, 703.68088633],
       [419.51935776, 158.85782918, 128.20024409,   0.        ,
        133.98905936, 267.62991551, 306.32295186, 575.48064224],
       [553.50841712, 292.84688854, 262.18930345, 133.98905936,
          0.        , 133.64085615, 172.3338925 , 441.49158288],
       [687.14927327, 426.48774469, 395.8301596 , 267.62991551,
        133.64085615,   0.        ,  38.69303635, 307.85072673],
       [725.84230962, 465.18078104, 434.52319595, 306.32295186,
        172.3338925 ,  38.69303635,   0.        , 269.15769038],
       [995.        , 734.33847143, 703.68088633, 575.48064224,
        441.49158288, 307.8507267

In [9]:
refspec = simsignals(H_RIOUX, 3)
refspec[:10]

[(260.66152857482973, 0.920443865947178),
 (291.3191136690316, 0.9152740673494263),
 (419.5193577561387, 1.1642820667033964),
 (292.8468885409387, 0.855243579015654),
 (426.4877446901903, 1.0651876493998582),
 (262.1893034467369, 0.9950358754480025),
 (434.5231959501799, 0.9201773503140263),
 (267.62991550888137, 0.9942165703492216),
 (306.3229518630728, 1.1700594579603005),
 (441.49158288423155, 0.8502854928575289)]

In [10]:
I_upper_sparse = sparse.COO(np.triu(I)).real
print(I_upper_sparse.todense())

[[0.00000000e+00 9.20443866e-01 9.15274067e-01 1.16428207e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  8.55243579e-01 1.06518765e+00 1.26375317e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.95035875e-01 6.08415874e-05 9.20177350e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.03839387e-06 9.94216570e-01 1.17005946e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 8.50285493e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.05946506e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.09024945e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]

In [11]:
E_upper_sparse = sparse.COO(np.triu(E_matrix))
print(E_upper_sparse.todense())

[[  0.         260.66152857 291.31911367 419.51935776 553.50841712
  687.14927327 725.84230962 995.        ]
 [  0.           0.          30.65758509 158.85782918 292.84688854
  426.48774469 465.18078104 734.33847143]
 [  0.           0.           0.         128.20024409 262.18930345
  395.8301596  434.52319595 703.68088633]
 [  0.           0.           0.           0.         133.98905936
  267.62991551 306.32295186 575.48064224]
 [  0.           0.           0.           0.           0.
  133.64085615 172.3338925  441.49158288]
 [  0.           0.           0.           0.           0.
    0.          38.69303635 307.85072673]
 [  0.           0.           0.           0.           0.
    0.           0.         269.15769038]
 [  0.           0.           0.           0.           0.
    0.           0.           0.        ]]


In [12]:
combo = sparse.stack([I_upper_sparse, E_upper_sparse])
combo.shape

(2, 8, 8)

In [13]:
iv = combo.reshape((2, 64)).T
# [(v, i) for i, v in row for row in iv if i > 0.01]


In [14]:
[(row[1], row[0]) for row in iv if row[0] > 0.01]

[(260.66152857482973, 0.920443865947178),
 (291.3191136690316, 0.9152740673494261),
 (419.5193577561387, 1.1642820667033964),
 (292.84688854093866, 0.8552435790156537),
 (426.4877446901903, 1.0651876493998582),
 (262.18930344673686, 0.9950358754480023),
 (434.52319595017997, 0.9201773503140263),
 (267.62991550888137, 0.9942165703492218),
 (306.32295186307283, 1.1700594579603),
 (441.4915828842316, 0.850285492857529),
 (307.85072673497996, 1.059465061336477),
 (269.1576903807885, 1.0902494458059926)]

In [15]:
iv_csr = iv.tocsr()
# iv_csr.todense()

In [16]:
iv_csr[iv_csr.getnnz(1)==2].todense()

matrix([[9.20443866e-01, 2.60661529e+02],
        [9.15274067e-01, 2.91319114e+02],
        [1.16428207e+00, 4.19519358e+02],
        [8.55243579e-01, 2.92846889e+02],
        [1.06518765e+00, 4.26487745e+02],
        [1.26375317e-05, 4.65180781e+02],
        [9.95035875e-01, 2.62189303e+02],
        [6.08415874e-05, 3.95830160e+02],
        [9.20177350e-01, 4.34523196e+02],
        [6.03839387e-06, 1.33989059e+02],
        [9.94216570e-01, 2.67629916e+02],
        [1.17005946e+00, 3.06322952e+02],
        [8.50285493e-01, 4.41491583e+02],
        [1.05946506e+00, 3.07850727e+02],
        [1.09024945e+00, 2.69157690e+02]])

In [17]:
# iv[iv.all(axis=1)==True].todense()

In [18]:
testspec = old_compile_spectrum(I, E)
assert np.allclose(refspec, testspec)

In [19]:
def compile_spectrum(I, E):
    I_upper_sparse = sparse.COO(np.triu(I))
    E_matrix = np.abs(E[:, np.newaxis] - E)
    E_upper_sparse = sparse.COO(np.triu(E_matrix))
    combo = sparse.stack([I_upper_sparse, E_upper_sparse])
    iv = combo.reshape((2, 64)).T
    iv_csr = iv.tocsr()
#     iv_filtered = iv_csr[iv_csr.getnnz(1)==2]
#     return [(row[1], row[0]) for row in iv if row[0] > 0.01]
    return iv_csr[iv_csr.getnnz(1)==2]

In [20]:
compilecsr = compile_spectrum(I, E)
compilespec = [(row[1], row[0]) for row in iv if row[0] > 0.01]
assert np.allclose(refspec, compilespec)

In [21]:
print(refspec[:10])
print(compilespec[:10])

[(260.66152857482973, 0.920443865947178), (291.3191136690316, 0.9152740673494263), (419.5193577561387, 1.1642820667033964), (292.8468885409387, 0.855243579015654), (426.4877446901903, 1.0651876493998582), (262.1893034467369, 0.9950358754480025), (434.5231959501799, 0.9201773503140263), (267.62991550888137, 0.9942165703492216), (306.3229518630728, 1.1700594579603005), (441.49158288423155, 0.8502854928575289)]
[(260.66152857482973, 0.920443865947178), (291.3191136690316, 0.9152740673494261), (419.5193577561387, 1.1642820667033964), (292.84688854093866, 0.8552435790156537), (426.4877446901903, 1.0651876493998582), (262.18930344673686, 0.9950358754480023), (434.52319595017997, 0.9201773503140263), (267.62991550888137, 0.9942165703492218), (306.32295186307283, 1.1700594579603), (441.4915828842316, 0.850285492857529)]


In [22]:
%timeit old_compile_spectrum(I, E)

18.6 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
%timeit compile_spectrum(I, E)

663 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%load_ext line_profiler


In [25]:
%lprun -f compile_spectrum compile_spectrum(I, E)

Timer unit: 1e-07 s

Total time: 0.0020156 s
File: <ipython-input-19-06af6766613f>
Function: compile_spectrum at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def compile_spectrum(I, E):
     2         1       4005.0   4005.0     19.9      I_upper_sparse = sparse.COO(np.triu(I))
     3         1        143.0    143.0      0.7      E_matrix = np.abs(E[:, np.newaxis] - E)
     4         1       3142.0   3142.0     15.6      E_upper_sparse = sparse.COO(np.triu(E_matrix))
     5         1       2109.0   2109.0     10.5      combo = sparse.stack([I_upper_sparse, E_upper_sparse])
     6         1       3175.0   3175.0     15.8      iv = combo.reshape((2, 64)).T
     7         1       2475.0   2475.0     12.3      iv_csr = iv.tocsr()
     8                                           #     iv_filtered = iv_csr[iv_csr.getnnz(1)==2]
     9                                           #     return [(row[1], row[0]) for row in i

this suggests the time required to convert to sparse alone makes this not worth it.

In [53]:
def dense_spectrum(I, E):
    I_upper = np.triu(I)
    E_matrix = np.abs(E[:, np.newaxis] - E)
    E_upper = np.triu(E_matrix)
    combo = np.stack([E_upper, I_upper])
#     combo = np.stack([I, E_matrix])
    iv = combo.reshape(2, I.shape[0]**2).T
#     iv_csr = iv.tocsr()
#     iv_filtered = iv_csr[iv_csr.getnnz(1)==2]
    
#     return iv_csr[iv_csr.getnnz(1)==2]
#     print((iv==0).any(1))
#     return [(row[1], row[0]) for row in iv if row[0] > 0.01]
#     return iv[iv[1]>0.01, :]
    return iv[iv[:, 1]>=0.01]

In [72]:
densespec = dense_spectrum(I, E)
print(len(densespec), len(refspec))
print(densespec[0], densespec[11], refspec[0], refspec[-1])
print(densespec[:12])
print(refspec)
assert np.allclose(refspec, densespec[:12])

10976 12
(641.5538518508765, 1.8516750575308343) (2275.282262973028, 1.012514628635217) (260.66152857482973, 0.920443865947178) (269.1576903807885, 1.0902494458059926)
[(641.5538518508765, 1.8516750575308343), (870.2287274432629, 2.079426394638906), (995.3138581296325, 2.056150647035006), (1000.0000000000009, 4.0000000000000036), (2275.2772661397066, 1.0125157797768498), (641.6326956617213, 1.8534015597641755), (874.9063460501102, 1.9338840313711023), (900.5809042168894, 0.051966743572861726), (998.9880757452315, 1.8589933743274527), (999.9999999999959, 3.9999999999999893), (1013.1752409790483, 0.13996461533612228), (2275.282262973028, 1.012514628635217)]
[(260.66152857482973, 0.920443865947178), (291.3191136690316, 0.9152740673494263), (419.5193577561387, 1.1642820667033964), (292.8468885409387, 0.855243579015654), (426.4877446901903, 1.0651876493998582), (262.1893034467369, 0.9950358754480025), (434.5231959501799, 0.9201773503140263), (267.62991550888137, 0.9942165703492216), (306.32

AssertionError: 

In [50]:
print(densespec[:10])

[(260.66152857482973, 0.920443865947178), (291.3191136690316, 0.9152740673494261), (419.5193577561387, 1.1642820667033964), (260.66152857482973, 0.920443865947178), (292.84688854093866, 0.8552435790156537), (426.4877446901903, 1.0651876493998582), (291.3191136690316, 0.9152740673494261), (262.18930344673686, 0.9950358754480023), (434.52319595017997, 0.9201773503140263), (419.5193577561387, 1.1642820667033964)]


In [27]:
%timeit dense_spectrum(I, E)

35.7 µs ± 922 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [54]:
%lprun -f dense_spectrum dense_spectrum(I, E)

Timer unit: 1e-07 s

Total time: 0.113472 s
File: <ipython-input-53-a505d57756d4>
Function: dense_spectrum at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def dense_spectrum(I, E):
     2         1     201575.0 201575.0     17.8      I_upper = np.triu(I)
     3         1     295616.0 295616.0     26.1      E_matrix = np.abs(E[:, np.newaxis] - E)
     4         1     336009.0 336009.0     29.6      E_upper = np.triu(E_matrix)
     5         1     164545.0 164545.0     14.5      combo = np.stack([E_upper, I_upper])
     6                                           #     combo = np.stack([I, E_matrix])
     7         1        129.0    129.0      0.0      iv = combo.reshape(2, I.shape[0]**2).T
     8                                           #     iv_csr = iv.tocsr()
     9                                           #     iv_filtered = iv_csr[iv_csr.getnnz(1)==2]
    10                                               
 

In [55]:
%lprun -f old_compile_spectrum old_compile_spectrum(I, E)

Timer unit: 1e-07 s

Total time: 2.16862 s
File: E:\Geoffrey\Documents\GitHub\cythontest\tests\test_simsignals.py
Function: old_compile_spectrum at line 231

Line #      Hits         Time  Per Hit   % Time  Line Contents
   231                                           def old_compile_spectrum(I, E):
   232         1         15.0     15.0      0.0      spectrum = []
   233         1         20.0     20.0      0.0      m = I.shape[0]
   234      2048       7811.0      3.8      0.0      for i in range(m - 1):
   235   2098175    7930363.0      3.8     36.6          for j in range(i + 1, m):
   236   2096128   13560384.0      6.5     62.5              if I[i, j] > 0.01:  # consider making this minimum intensity
   237                                                           # cutoff a function arg, for flexibility
   238     10976     110674.0     10.1      0.5                  v = abs(E[i] - E[j])
   239     10976      76937.0      7.0      0.4                  spectrum.append((v, I[i, 

In [30]:
def loop_fn(f, n):
    for i in range(n):
        res = f(I, E)
    return res

In [31]:
%lprun -f old_compile_spectrum loop_fn(old_compile_spectrum, 1)

Timer unit: 1e-07 s

Total time: 6.76e-05 s
File: E:\Geoffrey\Documents\GitHub\cythontest\tests\test_simsignals.py
Function: old_compile_spectrum at line 231

Line #      Hits         Time  Per Hit   % Time  Line Contents
   231                                           def old_compile_spectrum(I, E):
   232         1         14.0     14.0      2.1      spectrum = []
   233         1         29.0     29.0      4.3      m = I.shape[0]
   234         8         34.0      4.2      5.0      for i in range(m - 1):
   235        35        155.0      4.4     22.9          for j in range(i + 1, m):
   236        28        235.0      8.4     34.8              if I[i, j] > 0.01:  # consider making this minimum intensity
   237                                                           # cutoff a function arg, for flexibility
   238        12        124.0     10.3     18.3                  v = abs(E[i] - E[j])
   239        12         81.0      6.8     12.0                  spectrum.append((v, I[i,

In [32]:
%lprun -f dense_spectrum loop_fn(dense_spectrum, 1)

Timer unit: 1e-07 s

Total time: 0.0001632 s
File: <ipython-input-26-492a18f2d849>
Function: dense_spectrum at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents

In [ ]:
%lprun -f compile_spectrum loop_fn(compile_spectrum, 100)

In [33]:
from tests.test_simsignals import H11_NDARRAY

In [34]:
I, E = intensity_and_energy(H11_NDARRAY, 11)

In [35]:
I.shape

(2048, 2048)

In [51]:
%timeit old_compile_spectrum(I, E)

530 ms ± 7.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%timeit dense_spectrum(I, E)

109 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
old = old_compile_spectrum(I, E)
new = dense_spectrum(I, E)
print(old[:10], old[-10:])
print(new[:10], new[-10:])

[(641.5538518508765, 1.8516750575308343), (870.2287274432629, 2.079426394638906), (995.3138581296325, 2.056150647035006), (1000.0000000000009, 4.0000000000000036), (2275.2772661397066, 1.0125157797768498), (641.6326956617213, 1.8534015597641755), (874.9063460501102, 1.9338840313711023), (900.5809042168894, 0.051966743572861726), (998.9880757452315, 1.8589933743274527), (999.9999999999959, 3.9999999999999893)] [(888.9363123858448, 0.9649752281691011), (652.0985209252585, 0.9867131557430264), (879.9396662232512, 1.047106842934071), (631.7992526752951, 0.053606741633348165), (859.7295931687131, 0.08188013178406538), (662.106651895333, 2.1547646154616067), (1004.8159138628343, 1.9471311821109756), (1000.0000000000018, 4.0000000000000036), (889.9817108887182, 1.9115751136490524), (662.0513703953002, 2.15350676648617)]
[[6.41553852e+02 1.85167506e+00]
 [8.70228727e+02 2.07942639e+00]
 [9.95313858e+02 2.05615065e+00]
 [1.00000000e+03 4.00000000e+00]
 [2.27527727e+03 1.01251578e+00]
 [6.416326

In [48]:
def tupleize(array):
    newtuple = []
    for row in array:
        newtuple.append(tuple(row))
    return newtuple

In [49]:
newtuple = tupleize(new)
assert np.allclose(old, newtuple)

In [50]:
for row in old[:10]:
    print(row)
newtuple[:10]

(641.5538518508765, 1.8516750575308343)
(870.2287274432629, 2.079426394638906)
(995.3138581296325, 2.056150647035006)
(1000.0000000000009, 4.0000000000000036)
(2275.2772661397066, 1.0125157797768498)
(641.6326956617213, 1.8534015597641755)
(874.9063460501102, 1.9338840313711023)
(900.5809042168894, 0.051966743572861726)
(998.9880757452315, 1.8589933743274527)
(999.9999999999959, 3.9999999999999893)


[(641.5538518508765, 1.8516750575308343),
 (870.2287274432629, 2.079426394638906),
 (995.3138581296325, 2.056150647035006),
 (1000.0000000000009, 4.0000000000000036),
 (2275.2772661397066, 1.0125157797768498),
 (641.6326956617213, 1.8534015597641755),
 (874.9063460501102, 1.9338840313711023),
 (900.5809042168894, 0.051966743572861726),
 (998.9880757452315, 1.8589933743274527),
 (999.9999999999959, 3.9999999999999893)]

In [41]:
test = dense_spectrum(*intensity_and_energy(H_RIOUX_SPARSE, 3))
print(test)


[[260.66152857   0.92044387]
 [291.31911367   0.91527407]
 [419.51935776   1.16428207]
 [292.84688854   0.85524358]
 [426.48774469   1.06518765]
 [262.18930345   0.99503588]
 [434.52319595   0.92017735]
 [267.62991551   0.99421657]
 [306.32295186   1.17005946]
 [441.49158288   0.85028549]
 [307.85072673   1.05946506]
 [269.15769038   1.09024945]]
